# Web Scrapping Assignment 3

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import urllib.request
import os
from PIL import Image
import io
import re 
import time
import pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import NoSuchElementException
import warnings
warnings.filterwarnings("ignore")

# Q1. Answer:- A python  program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user.

In [2]:
driver=webdriver.Chrome('chromedriver.exe')
url=("https://www.amazon.in/")
driver.get(url)
driver.maximize_window()
time.sleep(2)

In [3]:
search_product=input("Enter the Product Name: ")

Enter the Product Name: laptop


In [4]:
print("Product to search is: ",search_product)

Product to search is:  laptop


In [5]:
product=driver.find_element(By.ID,"twotabsearchtextbox").send_keys(search_product)
search_product=driver.find_element(By.ID,"nav-search-submit-button").click()

# Q2. Answer:- A python program to scrape the details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”.

In [15]:
#fetching URL to collect all the Product URLS 
product_url=[]
start=0
end=2
for page in range(start,end):
    url=driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for i in url:
        product_url.append(i.get_attribute('href'))
    #next_button=driver.find_elements("xpath",'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]').click()
    next_4=driver.find_element("xpath",'/html/body/div[1]/div[2]/div[1]/div[1]/div/span[3]/div[2]/div[28]/div/div/span/span[1]').click()

In [16]:
len(product_url)

44

In [18]:
#Creating empty list 
Brand_name= []
Product_name = []
price=[]
Return_exc = []
Expected_Delivery = []
Availability = []

for i in product_url:
    driver.get(i)
    time.sleep(2)
    
    #Fetching data for Brand name    
    try:
        Brand=driver.find_element(By.XPATH,"//span[@class='a-size-large product-title-word-break']")
        Brand_name.append(Brand.text.split()[0]+" "+Brand.text.split()[1])
    except NoSuchElementException as e:
        Brand_name.append('-')
        
    #Fetching data for Product name        
    try:
        Product=driver.find_element(By.XPATH,"//span[@class='a-size-large product-title-word-break']")
        Product_name.append(Product.text)
    except NoSuchElementException as e:
        Product_name.append('-')
        
    #Fetching data for Product's Price
    try:
        price_tag=driver.find_element(By.XPATH,"//span[@class='a-price aok-align-center reinventPricePriceToPayMargin priceToPay']")
        price.append(price_tag.text.split("\n")[0])
    except NoSuchElementException as e:
        price.append('-')

    #Fetching data for Return/Exchange 
    try:
        Return_tag=driver.find_element(By.XPATH,"//div[@id='RETURNS_POLICY']/span/div[2]/a")
        Return_exc.append(Return_tag.text)
    except NoSuchElementException as e:
        Return_exc.append('-')

    #Fetching data for Expected Delivery 
    try:
        Delivery=driver.find_element(By.XPATH,"//div[@id='mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE']")
        Expected_Delivery.append(Delivery.text.split('delivery')[1].replace("Details"," "))
    except NoSuchElementException as e:
        Expected_Delivery.append('-')

    #Fetching data for Expected Delivery 
    try:
        Available=driver.find_element(By.XPATH,"//div[@id='availability']")
        Availability.append(Available.text)
    except NoSuchElementException as e:
        Availability.append('-')

In [19]:
print(len(Brand_name),len(Product_name),len(price),len(Return_exc),len(Expected_Delivery),len(Availability),len(product_url))

44 44 44 44 44 44 44


In [20]:
products=pd.DataFrame({"Brand_name":Brand_name,"Product Name":Product_name,"Product Price":price,"Return/Exchange":Return_exc,
                       "Expected Delivery":Expected_Delivery,"Availability":Availability, "Product URL":product_url})
print(products)

#saving to csv file
laptop=products.to_csv('laptop.csv')
laptop

,Brand_name,Product Name,Product Price,Return/Exchange,Expected Delivery,Availability,Product URL
0,Lenovo Ideapad,Lenovo Ideapad Slim 5 Pro AMD Ryzen 7 5800U 14...,"₹82,999",7 Days Replacement,"Tuesday, 2 August.",Usually dispatched in 2 to 3 days.,https://www.amazon.in/gp/slredirect/picassoRed...
1,Acer Travelmate,Acer Travelmate Business Laptop Intel Corei3 1...,"₹42,490",7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/gp/slredirect/picassoRed...
2,Lenovo IdeaPad,Lenovo IdeaPad Slim 5 Intel Core i5 11th Gen 1...,"₹58,990",7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/Lenovo-IdeaPad-Fingerpri...
3,"HP 14s,","HP 14s, AMD Ryzen 5-5500U 14 inches(35cm) FHD,...","₹46,928",7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/HP-Micro-Edge-Graphics-K...
4,"HP 14s,","HP 14s, 5th Gen AMD Ryzen 3- 8GB RAM/512GB SSD...",-,7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/HP-Micro-Edge-Keyboard-G...
5,Lenovo IdeaPad,Lenovo IdeaPad Slim 3 Intel Core i3 11th Gen 1...,"₹32,990",7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/Lenovo-IdeaPad-Warranty-...
6,"HP 14s,","HP 14s, 11th Gen Intel Core i3, 8GB RAM/512GB ...","₹39,154",7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/HP-Micro-Edge-Keyboard-G...
7,HP 14s,HP 14s 11th Gen Intel Core i3- 8GB RAM/256GB S...,"₹37,990",7 Days Replacement,"Friday, 29 July.",In stock.,https://www.amazon.in/HP-Micro-Edge-Anti-Glare...
8,HP 255,HP 255 G8 Laptop 3K9U1PA (AMD Ryzen 3-3300/4GB...,"₹30,000",7 Days Replacement,"Monday, 1 August. Order within 10 hrs 46 mins.",Only 1 left in stock.,https://www.amazon.in/HP-Laptop-3K9U1PA-Window...
9,HP 15-,HP 15- AMD Ryzen 3-3250U 15.6 inch(39.6 cm) FH...,-,7 Days Replacement,"Thursday, 28 July.",In stock.,https://www.amazon.in/HP-3-3250-Laptop-Windows...


# Q3. Answer:- A python program  to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [136]:
# Opening chrome browser
driver3 = webdriver.Chrome("chromedriver.exe")

# maximizing the browser size
driver.maximize_window()

# getting website "images.google.com"
url = "https://images.google.com/"
time.sleep(2)
    
#Creating empty list and giving search items as list and creating loop
urls = []    
data = []
search_item = ["fruits", "cars", "Machine Learning","guitar","cakes"]
for item in search_item:
    driver3.get(url)  
    time.sleep(5)
    search_img=driver3.find_element(By.XPATH,'//input[@class="gLFyf gsfi"]').send_keys(item)  #sending key word for search item
    
    search_btn = driver3.find_element(By.XPATH,'//button[@class="Tg7LZd"]').submit() #Clicking on search button
    
    # scrolling the web page to get more images
    for _ in range(20):
        driver3.execute_script("window.scrollBy(0,1000)")
        
        imgs = driver.find_elements(By.XPATH,"//img[@class='rg_i Q4LuWd']")
    img_url = []
    for image in imgs:
        source = image.get_attribute('src')
        if source is not None:
                if(source[0:4] == 'http'):
                    img_url.append(source)
    for i in img_url[:10]:
        urls.append(i)
                    
for i in range(len(urls)):
    if i > 10:
        break
    print("Downloading {0} of {1} images" .format(i, 10))
    response = requests.get(urls[i])
file = open(r"C:/Users/Archana/Flip_robo"+str(i)+".jpg", "wb")
file.write(response.content)
    

15362

# Q.4.Answer: A python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [36]:
driver4=webdriver.Chrome('chromedriver.exe')
driver4.get("https://www.flipkart.com/")
time.sleep(2)
driver4.maximize_window()
pop_close = driver4.find_element("xpath","/html/body/div[2]/div/div/button").click()
product = input(" Enter the name of Smartphone that has to be searched : ")
search_product4=driver4.find_element(By.CLASS_NAME,'_3704LK').send_keys(product)

 Enter the name of Smartphone that has to be searched : Oneplus Nord


In [37]:
search_button4= driver4.find_element(By.CLASS_NAME,'L0Z3Pu').click()

In [38]:
  
# Fetching urls of phones coming on 1st page
page1_urls = []
urls = driver4.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
for url in urls:
    page1_urls.append(url.get_attribute("href"))

In [39]:
#  creating  empty  list
Brand = []
Phone_name = []
Colour = []
RAM= []
ROM = []
Primary_Camera= []
Secondary_Camera = []
Display_Size= []
Battery_Capacity = []
Price = []
URL = []

In [41]:
# Scraping data from each url of page 1
for url in page1_urls:
    driver4.get(url)              
    URL.append(url)                                                          
    time.sleep(2)
    
    
    #Clicking on read more button
    try:
        read_more = driver4.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')     
        read_more.click()
    except NoSuchElementException:
        print("Exception occured while moving to next page")
        
    
    #Scraping brand name of phone data
    try:
        brand_tags = driver4.find_element(By.XPATH,'//span[@class="B_NuCI"]')      
        Brand.append(brand_tags.text.split()[0])
    except NoSuchElementException:
        Brand.append('-')
    
    
    #Scraping phone name data
    try:
        name_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')     
        Phone_name.append(name_tags.text)
    except NoSuchElementException:
        Phone_name.append('-')
    
    
    #Scraping phone color data
    try:
        color_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')      
        Colour.append(color_tags.text)
    except NoSuchElementException:
        Colour.append('-')
     
    
    #Scraping RAM data
    try:
        ram_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[2]/td[2]/ul/li')                
        RAM.append(ram_tags.text)
    except NoSuchElementException:
        RAM.append('-')
    
    
    #Scraping ROM data
    try:
        rom_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table[1]/tbody/tr[1]/td[2]/ul/li')        
        ROM.append(rom_tags.text)
    except NoSuchElementException:
        ROM.append('-')
        
        
    #Scraping Primary camera data
    try:                                                                                    
        pri_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[2]/td[2]/ul/li')
        Primary_Camera.append(pri_tags.text)
    except NoSuchElementException:
        Primary_Camera.append('-')
        
        
    #Scraping secondary camera data
    try:                                                                                    
        sec_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[1]')
        if sec_tags != "Secondary Camera" : 
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[1]').text == "Secondary Camera":
                sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[5]/td[2]/ul/li')
            else :
                raise NoSuchElementException
        else :
            sec_cam = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table[1]/tbody/tr[6]/td[2]/ul/li')
        Secondary_Camera.append(sec_cam.text)
    except NoSuchElementException:
        Secondary_Camera.append('-')
        
     #Scraping Display size data 
    try:
        disp_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/div')
        if disp_tags.text != "Displa Size" : raise NoSuchElementException
        disp_size = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table[1]/tbody/tr[1]/td[2]/ul/li')  
        Display_Size.append(disp_size.text)
    except NoSuchElementException:
        Display_Size.append('-')
        
        #Scraping battery capacity data
    try:
        if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/div').text != "Battery & Power Features" :
            if driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][9]/table/tbody/tr/td[2]/ul/li')                
            elif driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/div').text == "Battery & Power Features" :
                bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[1]')
                if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
                bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][8]/table/tbody/tr/td[2]/ul/li')
            else:
                raise NoSuchElementException
        else :
            bat_tags = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[1]')
            if bat_tags.text != "Battery Capacity" : raise NoSuchElementException
            bat_capa = driver4.find_element(By.XPATH,'//div[@class="_3k-BhJ"][10]/table/tbody/tr/td[2]/ul/li')              
        Battery_Capacity.append(bat_capa.text)
    except NoSuchElementException:
        Battery_Capacity.append('-')        
        
    #Scraping Price data
    try:
        price_tags = driver4.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')      
        Price.append(price_tags.text)
    except NoSuchElementException:
        Price.append('-')

In [42]:
smartphone=pd.DataFrame({"brand_name":Brand[:24],"smartphone_name":Phone_name[:24],"product_colour":Colour[:24],"product_RAM":RAM[:24],
                         "product_ROM":ROM[:24],"Primary_Camera":Primary_Camera[:24],"Secondary_Camera":Secondary_Camera[:24],
                         "Display_size":Display_Size[:24],"Battery_Capacity":Battery_Capacity[:24],"Phone_Price":Price[:24],"Product_URL":URL[:24]})

smartphone

,brand_name,smartphone_name,product_colour,product_RAM,product_ROM,Primary_Camera,Secondary_Camera,Display_size,Battery_Capacity,Phone_Price,Product_URL
0,OnePlus,Nord CE 2 Lite 5G,Blue Tide,,,,-,-,-,"₹19,795",https://www.flipkart.com/oneplus-nord-ce-2-lit...
1,OnePlus,Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,16MP Front Camera,-,-,-,"₹19,795",https://www.flipkart.com/oneplus-nord-ce-2-lit...
2,OnePlus,Nord 2T 5G,Gray Shadow,12 GB,256 GB,Primary Camera,-,-,-,"₹33,999",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
3,OnePlus,Nord 2T 5G,Jade Fog,12 GB,256 GB,Primary Camera,-,-,-,"₹33,999",https://www.flipkart.com/oneplus-nord-2t-5g-ja...
4,OnePlus,Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary Camera,-,-,-,"₹19,999",https://www.flipkart.com/oneplus-nord-ce-2-lit...
5,OnePlus,Nord CE 2 Lite 5G,Blue Tide,8 GB,128 GB,Primary Camera,-,-,-,"₹21,990",https://www.flipkart.com/oneplus-nord-ce-2-lit...
6,OnePlus,Nord 2T 5G,Gray Shadow,8 GB,128 GB,Primary Camera,-,-,-,"₹28,990",https://www.flipkart.com/oneplus-nord-2t-5g-gr...
7,OnePlus,Nord 2 5G,Blue Haze,8 GB,128 GB,Primary Camera,-,-,-,"₹29,125",https://www.flipkart.com/oneplus-nord-2-5g-blu...
8,OnePlus,Nord 2 5G,Gray Sierra,8 GB,128 GB,Primary Camera,-,-,-,"₹29,990",https://www.flipkart.com/oneplus-nord-2-5g-gra...
9,OnePlus,Nord CE 5G,Charcoal Ink,6 GB,128 GB,Primary Camera,-,-,-,"₹22,999",https://www.flipkart.com/oneplus-nord-ce-5g-ch...


In [43]:
smart_phone=smartphone.to_csv('smart_phone.csv')
smart_phone

# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [21]:
driver5=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening google maps web page
driver5.get("https://www.google.co.in/maps")
driver5.maximize_window()
time.sleep(2)

#Sending keyword for seach box and search button
city = input('Enter City name to search : ')
search_city = driver5.find_element(By.ID,"searchboxinput")                       
search_city.clear()                                                             
time.sleep(2)

search_city.send_keys(city)                                                     
search_button = driver5.find_element(By.ID,"searchbox-searchbutton").click()                                                             
time.sleep(3)
Location=driver5.current_url

try:
    coordinates=re.findall(r'@(.*)data',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
except IndexError as e:
    coordinates=re.findall(r'@(.*)z',Location)
    lat_lon=coordinates[0].split(",")
    lat=lat_lon[0]
    lon=lat_lon[1]
print("\nlatitude={}".format(lat))
print("\nlongitude={}".format(lon))

Enter City name to search : Delhi

latitude=28.6466772

longitude=76.8130617


# Q.6.Answer: Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21) from trak.in.

In [22]:
driver6=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)
# opening Track.in web page
driver6.get("https://trak.in/")
driver6.maximize_window()
time.sleep(2)

In [23]:
#click on funding deals and go to that page
fund_btn=driver6.find_element(By.XPATH,"//li[@id='menu-item-51510']/a").get_attribute('href')
driver6.get(fund_btn)

In [27]:
# Creating empty list
funding={}
Date=[]
Startup_name=[]
Industry_Vertical=[]
Sub_Vertical=[]
Location=[]
Investor=[]
Investment_Type=[]
Amount=[]

for i in range(54,57):
    driver6.find_element(By.XPATH,'//div[@id="tablepress-{}_wrapper"]/div/label/select/option[4]'.format(i)).click()

    # Scraping data of Date
    date_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[2]'.format(i))
    for d in date_tags:
        Date.append(d.text)

    # Scraping data of Startup Name
    name_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[3]'.format(i))
    for n in name_tags:
        Startup_name.append(n.text)
    
    # Scraping data of Industry OR Vertical
    ind_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[4]'.format(i))
    for v in ind_tags:
        Industry_Vertical.append(v.text)
    
    # Scraping data of Sub-Vertical
    sv_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[5]'.format(i))
    for s in sv_tags:
        Sub_Vertical.append(s.text)

    # Scraping data of Location
    loc_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[6]'.format(i))
    for loc in loc_tags:
        Location.append(loc.text)
    
    # Scraping data of Investor
    inv_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[7]'.format(i))
    for inv in inv_tags:
        Investor.append(inv.text)
        
    # Scraping data of Investment Type
    invt_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[8]'.format(i))
    for invt in invt_tags:
        Investment_Type.append(invt.text)
        
    # Scraping data of Amount
    amt_tags = driver6.find_elements(By.XPATH,'//table[@id="tablepress-{}"]/tbody/tr/td[9]'.format(i))
    for amt in amt_tags:
        Amount.append(amt.text)

In [28]:
funding=pd.DataFrame({"Date":Date,"Startup name":Startup_name,"Industry Vertical":Industry_Vertical,"Sub Vertical":Sub_Vertical,"Location":Location,"Investor":Investor,"Investment Type":Investment_Type,"Amount":Amount})
funding

,Date,Startup name,Industry Vertical,Sub Vertical,Location,Investor,Investment Type,Amount
0,15/01/2021,Digit Insurance,Financial Services,Insurance Services,Bengaluru,"A91 Partners, Faering Capital, TVS Capital Funds",Venture,"1,80,00,000"
1,28/01/2021,Bombay Shaving Company,Consumer Goods Company,"Shave care, beard care, and skincare products",New Delhi,Reckitt Benckiser,Venture,"6,172,258.50"
2,19/01/2021,DeHaat,AgriTech Startup,online marketplace for farm products and services,Patna,Prosus Ventures,Series C,"30,000,000"
3,19/01/2021,Darwinbox,SaaS,HR Tech,Mumbai,Salesforce Ventures,Seed,"15,000,000"
4,18/01/2021,mfine,Health Tech Startup,AI-powered telemedicine mobile app,Bengaluru,Heritas Capital Management,Venture Round,"16,000,000"
5,18/01/2021,Udayy,EdTech,Online learning platform for kids in class 1-5,Gurgaon,Sequoia Capital,Seed Funding,"15,000,000"
6,11/01/2021,True Elements,Food Startup,Whole Food plant based Nashta,Pune,SIDBI Venture Capital,Series,"100,000,000"
7,13/01/2021,Saveo,B2B E-commerce,Pharmacies,Bengaluru,"Matrix Partners India, RTP Global, others",Seed,"4,000,000"
8,11/02/2021,Doubtnut,Edu Tech,E-Learning Platform,Gurgaon,"SIG Global, Sequoia Capital, WaterBridge Ventu...",Series B,"2,500,000"
9,22/02/2021,Zomato,Hospitality,Online Food Delivery Platform,Gurgaon,"Tiger Global, Kora",Venture,"250,000,000"


# Q.7.Answer: Write a program to scrap all the available details of best gaming laptops from digit.in.

In [29]:
driver7=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening Digit.in web page
driver7.get("https://digit.in/")
driver7.maximize_window()
time.sleep(2)

In [30]:
driver7.find_element(By.XPATH,"/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a").click()

In [31]:
product_name=[]
seller=[]
price=[]

product_tag=driver7.find_elements(By.XPATH,"//*[@id='summtable']/tbody/tr/td[1]")
for i in product_tag:
    try:
        product_name.append(i.text)
    except NoSuchElementException:
        product_name.append("-")

seller_tag=driver7.find_elements(By.CLASS_NAME,"smmerchant")
for i in seller_tag:
    try:
        seller.append(i.text)
    except NoSuchElementException:
        seller.append("-")

price_tag=driver7.find_elements(By.CLASS_NAME,"smprice")
for i in price_tag:
    try:
        price.append(i.text)
    except NoSuchElementException:
        price.append("-")

Gaming=pd.DataFrame({"Product Name":product_name,"Seller":seller,"Price":price})
Gaming

,Product Name,Seller,Price
0,MSI Titan GT77-12UHS,N/A,N/A
1,Alienware x17 R2,Dell,"₹ 389,990"
2,Acer Predator Triton 500 SE PT516-52s,N/A,"₹ 300,000"
3,Omen by HP (16-B1371TX),Amazon,"₹ 170,990"
4,Acer Predator Helios 300 AN515-45 (NH.QBRSI.0,N/A,"₹ 172,999"
5,MSI Delta 15 (A5EFK-083IN),N/A,"₹ 188,990"
6,Omen by HP (16-c0141AX),Amazon,"₹ 129,899"
7,Lenovo Legion 5i Pro (82RF00MGIN),N/A,"₹ 230,890"
8,Alienware m15 R5 Ryzen Edition icc-c780001win,Dell,"₹ 144,990"
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Croma,"₹ 142,990"


# Q.8. Answer: Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [82]:
driver8=webdriver.Chrome("chromedriver.exe") 
time.sleep(2)

# opening web page
driver8.get("https://www.forbes.com/billionaires/")
driver8.maximize_window()
time.sleep(2)

In [96]:
# Create Empty list
rank = []
name = []
net_worth = []
age = []
citzn = []
source = []
industry = []

#Fetching Rank
rank_tag = driver8.find_elements(By.XPATH,"//div[@class='rank']")
for r in rank_tag:
    try:
        rank.append(r.text)
    except NoSuchElementException:
        rank.append("-")
    
#Fetching Name
name_tag = driver8.find_elements(By.XPATH,"//div[@class='personName']")
for n in name_tag:
    try:
        name.append(n.text)
    except NoSuchElementException:
        name.append("-")
        
#Fetching Net Worth
netwrth_tag = driver8.find_elements(By.XPATH,"//div[@class='netWorth']")
for nt in netwrth_tag:
    try:
        net_worth.append(nt.text)
    except NoSuchElementException:
        net_worth.append("-")
        
#Fetching Age
age_tag = driver8.find_elements(By.XPATH,"//div[@class='age']")
for a in age_tag:
    try:
        age.append(a.text)
    except NoSuchElementException:
        age.append("-")
    
#Fetching Citizenship
cit_tag = driver8.find_elements(By.XPATH,"//div[@class='countryOfCitizenship']")
for c in cit_tag:
    try:
        citzn.append(c.text)
    except NoSuchElementException:
        citzn.append("-")
    
#Fetching Source
src_tag = driver8.find_elements(By.XPATH,"//span[@class='source-text']")
for s in src_tag:
    try:
        source.append(s.text)
    except NoSuchElementException:
        source.append("-")
    
#Fetching Industry
ind_tag = driver8.find_elements(By.XPATH,"//div[@class='category']")
for ind in ind_tag:
        try:
            industry.append(ind.text)
        except NoSuchElementException:
            industry.append("-")
        


In [97]:
# Length of all the coloumns
len(rank),len(name),len(net_worth),len(age),len(citzn),len(source),len(industry)

(200, 200, 200, 200, 200, 200, 200)

In [103]:
#create Dataframe
Billionaires =pd.DataFrame({'Rank':rank,'Name':name,'Net Worth':net_worth,'Age': age,'Citizenship/Country':citzn,'Source':source,'Industry':industry})
Billionaires

,Rank,Name,Net Worth,Age,Citizenship/Country,Source,Industry
0,1.,Elon Musk,$219 B,,United States,,
1,2.,Jeff Bezos,$171 B,,United States,,
2,3.,Bernard Arnault & family,$158 B,,France,,
3,4.,Bill Gates,$129 B,,United States,,
4,5.,Warren Buffett,$118 B,,United States,,
...,...,...,...,...,...,...,...
195,192.,Marcel Herrmann Telles,$10.3 B,,Brazil,beer,
196,197.,Leon Black,$10 B,,United States,private equity,
197,197.,Joe Gebbia,$10 B,,United States,Airbnb,
198,197.,David Geffen,$10 B,,United States,"movies, record labels",


# Q.9. Answer: Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [107]:
driver9=webdriver.Chrome('chromedriver.exe')
driver9.get("https://www.youtube.com/watch?v=USccSZnS8MQ")
driver9.maximize_window()
time.sleep(2)

In [122]:
i=0
while(i<100):
    driver9.execute_script("window.scrollBy(0,500)") # scroll down to get more comments
    i+=1
while(i<402):
    driver9.execute_script("window.scrollBy(0,5000)") # scroll down to get more comments
    i+=1
    
comment = []
upvote = []
comment_time = []

comment_tag=(driver9.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]'))
for i in comment_tag:             
    try:
        comment.append(i.text)
    except NoSuchElementException:
        comment.append("-")

upvote_tag=(driver9.find_elements(By.XPATH,"//*[@id='vote-count-middle']"))
for i in upvote_tag:             
    try:
        upvote.append(i.text)
    except NoSuchElementException:
        upvote.append("-")

comment_time_tag=(driver9.find_elements(By.XPATH,"//*[@id='header-author']/yt-formatted-string/a"))
for i in comment_time_tag:             
    try:
        comment_time.append(i.text)
    except NoSuchElementException:
        comment_time.append("-")

In [123]:
video=pd.DataFrame({"Comment":comment,"Upvote":upvote,"Comment_ Time":comment_time})
video

,Comment,Upvote,Comment_ Time
0,Lose yourself in the Magic of “Love” with Sidh...,34K,1 year ago
1,"Hi. I'm an American singer, Jason Ray. The sto...",58K,1 year ago
2,Still Bollywood industry have gems like Stebin...,34,1 month ago
3,The singer's singing style is quite like Atif ...,635,2 months ago
4,"Sidharth is so expressive, every time I see hi...",73,9 days ago (edited)
5,"A message to the future generation.""don't Let ...",132,2 months ago
6,Suddenly I remembered this song.. only somethi...,52,2 weeks ago
7,He is the only actor i think who challenges hi...,302,1 year ago (edited)
8,"I love how his voice is fricking deep , but hi...",88,2 months ago
9,For all the couples who are MADLY in LOVE. Thi...,128,2 months ago


# Q.10. Answer: Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [90]:
driver0=webdriver.Chrome('chromedriver.exe')
driver0.get("https://www.hostelworld.com/")
driver0.maximize_window()
time.sleep(2)

In [91]:
driver0.find_element(By.ID,'location-text-input-field').click()
time.sleep(2)
driver0.find_element(By.XPATH,'//input[@id="location-text-input-field"]').send_keys('London')
time.sleep(2)

In [95]:
driver0.find_element(By.XPATH,'//*[@id="predicted-search-results"]/li[1]').click()

In [96]:
driver0.find_element("xpath",'//div[@class="search-button"]').click()

In [106]:
# Make empty lists
Hostel_Name = []
Distance = []
overall_review = []
total_reviews = []
facilities = []
price = []
Rating = []
property_description = []

In [107]:
while(True):
    
    # Hostel name
    names = driver0.find_elements(By.XPATH,'//h2[@class="title title-6"]')
    for name in names:
        Hostel_Name.append(name.text)
    time.sleep(2)
        
    # Distance from city
    distance = driver0.find_elements(By.XPATH,'//span[@class="description"]')
    for d in distance:
        Distance.append(d.text)
    time.sleep(2)
        
    #Review    
    review = driver0.find_elements(By.XPATH,'//div[@class="keyword"]//span')
    for r in review:
        overall_review.append(r.text)
    time.sleep(2)
    
    # Total number of reviews     
    t_review = driver0.find_elements(By.XPATH,'//div[@class="reviews"]')
    for t in t_review:
        total_reviews.append(t.text)
    time.sleep(2)
    
    # facilities
    service = driver0.find_elements(By.XPATH,'//div[@class="facilities-label facilities"]')
    for s in service:
        facilities.append(s.text)
    time.sleep(2)
    
    # Prices    
    prices = driver0.find_elements(By.XPATH,'//div[@class="price-col"]')
    for p in prices:
        price.append(p.text)
    time.sleep(2)    
    try:
        next_button = driver0.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        next_button.click()
    except:
        break
        
time.sleep(2) 

# Separate  Privates_From price  and  Dorms_From price
private = []
for i in range(0,len(price),2):
    private.append(price[i])
time.sleep(2)

dorms = []
for i in range(1,len(price),2):
    dorms.append(price[i]) 

In [109]:
# scrape Hostels URL
hostel_url = []

while(True):
    urls = driver0.find_elements(By.XPATH,'//h2[@class="title title-6"]/a')
    for url in urls:
        hostel_url.append(url.get_attribute("href"))
    time.sleep(2)    
        
    try:
        next_button = driver0.find_element(By.XPATH,'//div[@class="pagination-item pagination-prev"]')
        next_button.click()
    except:
        break
        
Rate = []
for page in hostel_url:
    driver0.get(page)
    
    # Rating
    try:
        ratings = driver0.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[1]/div[1]/div[1]')
        Rate.append(ratings.text)
    except NoSuchElementException:
        Rate.append("No Rating")  
    time.sleep(2)
    
    # Property Description
    try:
        pd = driver0.find_element(By.XPATH,'//*[@id="__layout"]/div/div[1]/section/div[6]/div/div[2]/div[2]/div/div[2]')
        property_description.append(pd.text)
    except NoSuchElementException:
        property_description.append("No Description")  

    
time.sleep(2)        
# remove extra data from Rating     
all_text = []
for i in Rate:
    all_text.append(i.split())
time.sleep(2)

for i in all_text:
    Rating.append(i[0]) 

In [110]:
print(
    len(Hostel_Name),len(Distance),len(Rating),len(total_reviews),len(overall_review),
    len(private),len(dorms),len(facilities),len(property_description))

30 30 30 30 33 30 30 30 30


In [113]:
Hostel=pd.DataFrame({"Hostel Name":Hostel_Name,"Distance":Distance,"Rating":Rating,"Total Reviews":total_reviews,"Overall Review":overall_review[:30],
                     "Private Price":private,"Dorms Price":dorms,"Facilities":facilities,"Property Description":property_description})
Hostel

,Hostel Name,Distance,Rating,Total Reviews,Overall Review,Private Price,Dorms Price,Facilities,Property Description
0,Backpackers Heaven@New King,Hostel - 1.3km from city centre,8.7,1199 Total Reviews,Fabulous,Privates From\nRs2700 Rs945,Dorms From\nRs1330 Rs466,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
1,Yes Boss by Backpackers Heaven,Hostel - 1.2km from city centre,7.7,22 Total Reviews,Very Good,Privates From\nRs2600 Rs1430,Dorms From\nRs1300 Rs715,Free WiFi,No Description
2,Hindustan-by Backpackers Heaven,Hostel - 1.5km from city centre,8.3,569 Total Reviews,Fabulous,Privates From\nRs1782.33 Rs1141,Dorms From\nRs899 Rs575,Free WiFi,No Description
3,Joey's Hostel New Delhi,Hostel - 4.9km from city centre,9.0,1705 Total Reviews,Fabulous,No Privates Available,Dorms From\nRs654.3 Rs589,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
4,Hostel Smyle Inn,Hostel - 1.2km from city centre,8.9,1864 Total Reviews,Very Good,Privates From\nRs1147,No Dorms Available,Free WiFi\nFree Breakfast\nFollows Covid-19 sa...,No Description
5,Zostel Delhi,Hostel - 1.4km from city centre,7.9,1228 Total Reviews,Fabulous,Privates From\nRs2239,Dorms From\nRs615,Free WiFi\nFollows Covid-19 sanitation guidance,No Description
6,The Hosteller New Delhi,Hostel - 7.9km from city centre,7.8,115 Total Reviews,Superb,Privates From\nRs1860.33 Rs1395,No Dorms Available,Free WiFi,No Description
7,Moustache Delhi,Hostel - 8.9km from city centre,8.7,214 Total Reviews,Fabulous,Privates From\nRs3718.01 Rs2789,Dorms From\nRs931.67 Rs699,Free WiFi,No Description
8,Hotel Hari Piorko,Hostel - 1.2km from city centre,8.6,20 Total Reviews,Very Good,Privates From\nRs1400 Rs1302,Dorms From\nRs699 Rs650,Free WiFi,No Description
9,goStops Delhi,Hostel - 1.5km from city centre,7.3,781 Total Reviews,Very Good,No Privates Available,Dorms From\nRs1220.33 Rs732,Free WiFi\nFollows Covid-19 sanitation guidance,No Description


In [114]:
Hostels=Hostel.to_csv('Hostels.csv')
Hostels